<a href="https://colab.research.google.com/github/MyrtooS/Explainability-Methods-for-Brain-Tumor-Classification/blob/main/load_image_train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive'
folder='/content/drive/My Drive/MRI Classification/mri_kaggle'

Mounted at /content/drive
MyDrive  Shareddrives


In [ ]:
import cv2
import os
import glob
from PIL import Image
import os, os.path
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.datasets 
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import EfficientNetB0

1η Προσέγγιση - Κατηγοριοποίηση με 4 κλάσεις

Φόρτωση εικόνων

In [ ]:
list_of_files_train = []
path_train = "/content/drive/My Drive/MRI Classification/mri_kaggle/Training/"
for file in os.listdir(path_train): #reading all sub file directories
    mri_path_train= path_train+str(file)+'/*.jpg'
    list_of_files_train.append(mri_path_train)

print(list_of_files_train)


['/content/drive/My Drive/MRI Classification/mri_kaggle/Training/pituitary_tumor/*.jpg', '/content/drive/My Drive/MRI Classification/mri_kaggle/Training/glioma_tumor/*.jpg', '/content/drive/My Drive/MRI Classification/mri_kaggle/Training/meningioma_tumor/*.jpg', '/content/drive/My Drive/MRI Classification/mri_kaggle/Training/no_tumor/*.jpg']


In [ ]:
array_images_train = []
array_labels_train = []
    
label_counter_train=0
for i in list_of_files_train:
  images_train = glob.glob(i)
        
  for image in images_train:
    with open(image, 'rb') as r_image:
           
            
      img = Image.open(image)
      resized_image = img.resize((224,224))
      resized_image=np.array(resized_image)
                
      array_images_train.append(resized_image)
      array_labels_train.append(label_counter_train)  
  
  #the image labels will be 0,1,2,3:
  #0 is for pituitary tumor, 1 is for glioma tumor, 2 is for meningioma tumor and 3 is for no tumor

  label_counter_train = label_counter_train + 1 

In [ ]:
#converting list to array
x_train = np.array(array_images_train)
y_train = np.array(array_labels_train)

y_train = tensorflow.keras.utils.to_categorical(y_train, 4)



In [ ]:
y_train.shape

(4054, 4)

In [ ]:
y1_train = y_train.reshape(4054,-1)
x1_train = x_train.reshape(4054,-1)

In [ ]:


#oversampling the data because the 'no tumor' class did not have enough images
sm = SMOTE(random_state=42)
X_smote, y_smote = sm.fit_resample(x1_train, y1_train)
x_train  = X_smote.reshape(4524,224,224,3)

#spliting train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_smote, test_size=0.2, random_state=1, stratify = y_smote)

Τα 2 CNN μοντέλα θα ξεχωρίζονται με τα ονόματα gap και nogap κάτι το οποίο αναφέρεται στην ύπαρξη ή οχι GAP επιπέδου

1ο μοντέλο - CNN

In [ ]:
gap_model = tensorflow.keras.Sequential() 
gap_model.add(layers.Input(shape=x_train[0].shape))
gap_model.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.BatchNormalization())
gap_model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
gap_model.add(layers.Dropout(0.2))

gap_model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.BatchNormalization())
gap_model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
gap_model.add(layers.Dropout(0.2))

gap_model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.BatchNormalization())
gap_model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
gap_model.add(layers.Dropout(0.2))

gap_model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
gap_model.add(layers.BatchNormalization())
gap_model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
gap_model.add(layers.Dropout(0.2))

gap_model.add(layers.AveragePooling2D())
gap_model.add(layers.Flatten())
gap_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
X_train_gap = X_train.astype('float32') / 255.0
X_test_gap = X_test.astype('float32') / 255.0

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
gap_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
gap_model.fit(X_train_gap, Y_train, batch_size=32, epochs=13)

Epoch 1/13
114/114 [==============================] - 70s 449ms/step - loss: 1.0518 - accuracy: 0.6261
Epoch 2/13
114/114 [==============================] - 54s 471ms/step - loss: 0.8084 - accuracy: 0.6936
Epoch 3/13
114/114 [==============================] - 56s 493ms/step - loss: 0.6986 - accuracy: 0.7372
Epoch 4/13
114/114 [==============================] - 55s 485ms/step - loss: 0.6555 - accuracy: 0.7585
Epoch 5/13
114/114 [==============================] - 55s 486ms/step - loss: 0.5896 - accuracy: 0.7988
Epoch 6/13
114/114 [==============================] - 55s 486ms/step - loss: 0.4694 - accuracy: 0.8372
Epoch 7/13
114/114 [==============================] - 55s 485ms/step - loss: 0.4109 - accuracy: 0.8544
Epoch 8/13
114/114 [==============================] - 55s 485ms/step - loss: 0.3487 - accuracy: 0.8759
Epoch 9/13
114/114 [==============================] - 55s 486ms/step - loss: 0.3202 - accuracy: 0.8875
Epoch 10/13
114/114 [==============================] - 55s 485ms/step - l

In [ ]:
gap_model.save('/content/drive/My Drive/MRI Classification/gap_4.h5')

In [ ]:
rounded_predictions_gap = np.argmax(gap_model.predict(X_test_gap), axis=-1)
rounded_predictions_gap[1]
rounded_labels_gap=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_predictions_gap, rounded_predictions_gap)
print(cm)

In [ ]:
accuracy = accuracy_score(rounded_predictions_gap, rounded_predictions_gap)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions_gap, rounded_predictions_gap, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions_gap, rounded_predictions_gap,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions_gap, rounded_predictions_gap,average = None)
print('F1 score: ',f1)

2o μοντέλο - CNN με 1x1 Convolutions

In [ ]:
model_nogap = tensorflow.keras.Sequential() 
model_nogap.add(layers.Input(shape=x_train[0].shape))
model_nogap.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.BatchNormalization())
model_nogap.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))

model_nogap.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.BatchNormalization())
model_nogap.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model_nogap.add(layers.Dropout(0.2))


model_nogap.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.BatchNormalization())
model_nogap.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))


model_nogap.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
model_nogap.add(layers.BatchNormalization())
model_nogap.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model_nogap.add(layers.Dropout(0.2))

model_nogap.add(layers.Flatten())
model_nogap.add(layers.Dense(4096 , activation='relu'))
model_nogap.add(layers.Dense(1, activation='sigmoid'))
model_nogap.summary()

In [ ]:
X_train_nogap = X_train.astype('float32') / 255.0
X_test_nogap = X_test.astype('float32') / 255.0

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
model_nogap.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model_nogap.fit(X_train_nogap, Y_train, batch_size=32, epochs=13)

In [ ]:
score = model_nogap.evaluate(Y_train,Y_test)

In [ ]:
model_nogap.save('/content/drive/My Drive/MRI Classification/gap_4.h5')

In [ ]:
rounded_predictions_nogap = np.argmax(model_nogap.predict(X_test_nogap), axis=-1)
rounded_predictions_nogap[1]
rounded_labels_nogap=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels_nogap, rounded_predictions_nogap)
print(cm)

In [ ]:
accuracy = accuracy_score(rounded_predictions_nogap, rounded_labels_nogap)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions_nogap, rounded_labels_nogap, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions_nogap, rounded_labels_nogap,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions_nogap, rounded_labels_nogap,average = None)
print('F1 score: ',f1)

3ο μοντέλο - EfficientNetB0


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
efficientnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape= (224,224,3))

x = layers.Flatten()(efficientnet.output)
x = layers.Dense(units=4096,activation="relu")(x)
x = layers.Dense(units=4,name = "dense_final")(x)
x = layers.Activation("softmax")(x)

eff_model = tf.keras.Model(efficientnet.input, [x])

In [ ]:
eff_model.save('/content/drive/My Drive/MRI Classification/Multi_EfficientNet.h5')

In [ ]:
X_train_eff = tf.keras.applications.vgg16.preprocess_input(X_train, data_format=None)
X_test_eff = tf.keras.applications.vgg16.preprocess_input(X_test, data_format=None)

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
eff_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
eff_model.fit(X_train_eff, Y_train, batch_size=16, epochs=5)

In [ ]:
score = eff_model.evaluate(X_test_eff,Y_test)

In [ ]:
y = eff_model.get_layer('dense_final').output
y = tf.math.scalar_mul(0.2, y)
y = layers.Activation("sigmoid")(y)

eff_y_model = Model(inputs=eff_model.input, outputs=y)

In [ ]:
from sklearn.metrics import confusion_matrix
rounded_predictions_eff = np.argmax(eff_model.predict(X_test_eff), axis=-1)
rounded_predictions_eff[1]
rounded_labels_eff=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels_eff, rounded_predictions_eff)
print(cm)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(rounded_predictions_eff, rounded_labels_eff)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions_eff, rounded_labels_eff, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions_eff, rounded_labels_eff,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions_eff, rounded_labels_eff,average = None)
print('F1 score: ',f1)

4ο Μοντέλο -  VGG16

In [ ]:
vgg = VGG16(weights="imagenet", include_top=False, input_shape=(224,224,3))
vgg.trainable = False 
x = layers.Flatten()(vgg.output)
x1 = layers.Dense(units=4096,activation="relu")(x)
x2 = layers.Dense(units=4,name="dense_final")(x1)
x4 = layers.Activation("softmax")(x2)

vgg_model = tf.keras.Model(vgg.input, [x4])

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
X_train_vgg = tf.keras.applications.vgg16.preprocess_input(X_train, data_format=None)
X_test_vgg = tf.keras.applications.vgg16.preprocess_input(X_test, data_format=None)

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
vgg_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
vgg_model.fit(X_train_vgg, Y_train, batch_size=16, epochs=5)

In [ ]:
score = vgg_model.evaluate(X_test_vgg,Y_test)

In [ ]:
vgg_model.save('/content/drive/My Drive/MRI Classification/Multi_VGG16_1.h5')

In [ ]:
y = vgg_model.get_layer('dense_final').output
y = tf.math.scalar_mul(0.2, y)
y = layers.Activation("sigmoid")(y)

vgg_y_model = Model(inputs=vgg_model.input, outputs=y)

In [ ]:
from sklearn.metrics import confusion_matrix
rounded_predictions_vgg = np.argmax(vgg_model.predict(X_test_vgg), axis=-1)
rounded_predictions_vgg[1]
rounded_labels_vgg=np.argmax(Y_test, axis=1)

cm = confusion_matrix(rounded_labels_vgg, rounded_predictions_vgg)
print(cm)

[[222   1   1   2]
 [ 15 200   9   2]
 [ 33  34 148  12]
 [  7   4   1 214]]


In [ ]:
accuracy = accuracy_score(rounded_predictions_vgg, rounded_labels_vgg)
print('Accuracy: ' ,accuracy)
precision = precision_score(rounded_predictions_vgg, rounded_labels_vgg, average = None)
print('Precision: ' , precision)
recall = recall_score(rounded_predictions_vgg, rounded_labels_vgg,average = None)
print('Recall: ' ,recall)
f1 = f1_score(rounded_predictions_vgg, rounded_labels_vgg,average = None)
print('F1 score: ',f1)

Accuracy:  0.8662983425414365
Precision:  [0.98230088 0.88495575 0.65198238 0.94690265]
Recall:  [0.80144404 0.83682008 0.93081761 0.93043478]
F1 score:  [0.88270378 0.86021505 0.76683938 0.93859649]


2η Προσέγγιση - Δυαδική Κατηγοριοποίηση

In [ ]:
list_of_files_train_binary = []
path_train = "/content/drive/My Drive/MRI Classification/Brain Tumor Data Set/"
for file in os.listdir(path_train): #reading all sub file directories
    mri_path_train= path_train+str(file)+'/*.jpg'
    list_of_files_train_binary.append(mri_path_train)

print(list_of_files_train_binary)

['/content/drive/My Drive/MRI Classification/Brain Tumor Data Set/Healthy/*.jpg', '/content/drive/My Drive/MRI Classification/Brain Tumor Data Set/Brain Tumor/*.jpg', '/content/drive/My Drive/MRI Classification/Brain Tumor Data Set/TestImages/*.jpg', '/content/drive/My Drive/MRI Classification/Brain Tumor Data Set/TestImages.png/*.jpg', '/content/drive/My Drive/MRI Classification/Brain Tumor Data Set/TrainImages.png/*.jpg']


In [ ]:
array_images_train_binary = []
array_labels_train_binary = []
    
label_counter_train_binary=0
for i in list_of_files_train_binary:
  images_train_binary = glob.glob(i)
        
  for image in images_train_binary:
    with open(image, 'rb') as r_image:
           
            
      img = Image.open(image)
      resized_image = img.resize((224,224))
      resized_image=np.array(resized_image)
      if len(resized_image.shape) == 3:
        if resized_image.shape[0] == 224 and resized_image.shape[1] == 224 and resized_image.shape[2] == 3:    
                    
          array_images_train_binary.append(np.array(resized_image))
          array_labels_train_binary.append(label_counter_train_binary) 

      
  label_counter_train_binary = label_counter_train_binary + 1 


In [ ]:
x_train_binary = np.array(array_images_train_binary)
y_train_binary = np.array(array_labels_train_binary)

from sklearn.model_selection import train_test_split
X_train_binary, X_test_binary, Y_train_binary, Y_test_binary = train_test_split(x_train_binary, y_train_binary, test_size=0.2, random_state=1,stratify = y_train_binary)

1o μοντέλο - CNN

In [ ]:
model_gap_binary = tensorflow.keras.Sequential() 
model_gap_binary.add(layers.Input(shape=x_train_binary[0].shape))
model_gap_binary.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.BatchNormalization())
model_gap_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model_gap_binary.add(layers.Dropout(0.2))

model_gap_binary.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model_gap_binary.add(layers.Dropout(0.2))

model_gap_binary.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model_gap_binary.add(layers.Dropout(0.2))

model_gap_binary.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
model_gap_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model_gap_binary.add(layers.Dropout(0.2))

model_gap_binary.add(layers.AveragePooling2D())
model_gap_binary.add(layers.Flatten())
model_gap_binary.add(layers.Dense(1, activation="sigmoid"))
model_gap_binary.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 3)       84        
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 64)     256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 112, 112, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     7

In [ ]:
X_train_bgap = X_train_binary.astype('float32') / 255.0
X_test_bgap = X_test_binary.astype('float32') / 255.0

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.00001)
model_gap_binary.compile(optimizer=adam ,loss='binary_crossentropy', metrics=['accuracy'])
model_gap_binary.fit(X_train_bgap, Y_train_binary, batch_size=16, epochs=20)

Epoch 1/20
215/215 [==============================] - 64s 229ms/step - loss: 0.6432 - accuracy: 0.6351
Epoch 2/20
215/215 [==============================] - 48s 225ms/step - loss: 0.5881 - accuracy: 0.6977
Epoch 3/20
215/215 [==============================] - 49s 228ms/step - loss: 0.5104 - accuracy: 0.7515
Epoch 4/20
215/215 [==============================] - 49s 228ms/step - loss: 0.4269 - accuracy: 0.8086
Epoch 5/20
215/215 [==============================] - 49s 228ms/step - loss: 0.3710 - accuracy: 0.8339
Epoch 6/20
215/215 [==============================] - 49s 228ms/step - loss: 0.3234 - accuracy: 0.8560
Epoch 7/20
215/215 [==============================] - 49s 229ms/step - loss: 0.2730 - accuracy: 0.8781
Epoch 8/20
215/215 [==============================] - 49s 228ms/step - loss: 0.2609 - accuracy: 0.8918
Epoch 9/20
215/215 [==============================] - 49s 228ms/step - loss: 0.2236 - accuracy: 0.9118
Epoch 10/20
215/215 [==============================] - 49s 228ms/step - l

In [ ]:
score = model_gap_binary.evaluate(X_test_bgap,Y_test_binary)

27/27 [==============================] - 8s 216ms/step - loss: 0.2171 - accuracy: 0.9244


In [ ]:
model_gap_binary.save('/content/drive/My Drive/MRI Classification/GAP_Binary2.h5')

In [ ]:
y_pred_gap = model_gap_binary.predict(X_test_binary)
y_pred_gap = (y_pred_gap > 0.5)
  
cm = confusion_matrix(Y_test_binary, y_pred_gap)

print(cm)

[[382  24]
 [176 278]]


In [ ]:
accuracy = accuracy_score(y_pred_gap, Y_test_binary)
print('Accuracy: ' ,accuracy)
precision = precision_score(y_pred_gap, Y_test_binary, average = None)
print('Precision: ' , precision)
recall = recall_score(y_pred_gap, Y_test_binary,average = None)
print('Recall: ' ,recall)
f1 = f1_score(y_pred_gap, Y_test_binary,average = None)
print('F1 score: ',f1)

Accuracy:  0.7674418604651163
Precision:  [0.9408867 0.6123348]
Recall:  [0.68458781 0.9205298 ]
F1 score:  [0.79253112 0.73544974]


2o μοντέλο - CNN με 1x1 Convolutions 

In [ ]:
X_train_b_nogap = X_train_binary.astype('float32') / 255.0

In [ ]:
X_test_b_nogap = X_test_binary.astype('float32') / 255.0

In [ ]:
no_gap_model_binary = tensorflow.keras.Sequential() 
no_gap_model_binary.add(layers.Input(shape=x_train[0].shape))
no_gap_model_binary.add(layers.Conv2D(3, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(64, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.BatchNormalization())
no_gap_model_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))

no_gap_model_binary.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(128, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.BatchNormalization())
no_gap_model_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
no_gap_model_binary.add(layers.Dropout(0.2))


no_gap_model_binary.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(256, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.BatchNormalization())
no_gap_model_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))


no_gap_model_binary.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(512, (3,3), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.Conv2D(64, (1,1), strides = (1,1), padding="same", activation='relu'))
no_gap_model_binary.add(layers.BatchNormalization())
no_gap_model_binary.add(layers.MaxPooling2D(pool_size=(2, 2),strides=2))
no_gap_model_binary.add(layers.Dropout(0.2))

no_gap_model_binary.add(layers.Flatten())
no_gap_model_binary.add(layers.Dense(4096 , activation='relu'))
no_gap_model_binary.add(layers.Dense(1, activation='sigmoid'))
no_gap_model_binary.summary()

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.00001)
no_gap_model_binary.compile(optimizer=adam ,loss='binary_crossentropy', metrics=['accuracy'])
no_gap_model_binary.fit(X_train_b_nogap, Y_train, batch_size=32, epochs=8)

In [ ]:
no_gap_model_binary.save('/content/drive/My Drive/MRI Classification/Binary_Νο_GAP.h5')


In [ ]:
score = no_gap_model_binary.evaluate(X_test_b_nogap,Y_test_binary)

27/27 [==============================] - 4s 90ms/step - loss: 0.1280 - accuracy: 0.9558


In [ ]:
y_pred_nogap = no_gap_model_binary.predict(X_test_b_nogap)
y_pred_nogap = (y_pred_nogap[:,0] > 0.5)
    
cm = confusion_matrix(Y_test_binary, y_pred_nogap)

print(cm)

[[388  18]
 [ 20 434]]


In [ ]:
accuracy = accuracy_score(y_pred_nogap, Y_test_binary)
print('Accuracy: ' ,accuracy)
precision = precision_score(y_pred_nogap, Y_test_binary, average = None)
print('Precision: ' , precision)
recall = recall_score(y_pred_nogap, Y_test_binary,average = None)
print('Recall: ' ,recall)
f1 = f1_score(y_pred_nogap, Y_test_binary,average = None)
print('F1 score: ',f1)

Accuracy:  0.9558139534883721
Precision:  [0.95566502 0.95594714]
Recall:  [0.95098039 0.96017699]
F1 score:  [0.95331695 0.9580574 ]


3o μοντέλο - EfficientNetB0


In [ ]:

efficientnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape= (224,224,3))

x = layers.Flatten()(efficientnet.output)

x = layers.Dense(units=512,activation="relu")(x)

x = layers.Dense(units=1,name = "dense_final")(x)
x = layers.Activation("sigmoid")(x)

b_eff_model = tf.keras.Model(efficientnet.input, [x])

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
b_eff_model.compile(optimizer=adam ,loss='binary_crossentropy', metrics=['accuracy'])
b_eff_model.fit(X_train_binary, Y_train_binary, batch_size=16, epochs=5)

In [ ]:
b_eff_model.save('/content/drive/My Drive/MRI Classification/Binary_EfficientNetB0.h5')

In [ ]:
score = b_eff_model.evaluate(X_test_binary,Y_test_binary)

27/27 [==============================] - 4s 82ms/step - loss: 0.1369 - accuracy: 0.9616


In [ ]:
y = b_eff_model.get_layer('dense_final').output
y = tf.math.scalar_mul(0.2, y)
y = layers.Activation("sigmoid")(y)

b_eff_y_model = Model(inputs=b_eff_model.input, outputs=y)

In [ ]:
y_pred_eff = b_eff_model.predict(X_test_binary)
y_pred_eff = (y_pred_eff > 0.5)
    
cm = confusion_matrix(Y_test_binary, y_pred_eff)

print(cm)

[[406   0]
 [ 33 421]]


In [ ]:
accuracy = accuracy_score(y_pred_eff, Y_test_binary)
print('Accuracy: ' ,accuracy)
precision = precision_score(y_pred_eff, Y_test_binary, average = None)
print('Precision: ' , precision)
recall = recall_score(y_pred_eff, Y_test_binary,average = None)
print('Recall: ' ,recall)
f1 = f1_score(y_pred_eff, Y_test_binary,average = None)
print('F1 score: ',f1)

Accuracy:  0.9616279069767442
Precision:  [1.         0.92731278]
Recall:  [0.92482916 1.        ]
F1 score:  [0.96094675 0.96228571]


4ο μοντέλο - VGG16

In [ ]:
b_vgg =  VGG16(weights='imagenet',include_top=False,input_shape= (224,224,3))

x = layers.Flatten()(b_vgg.output)

x = layers.Dense(units=512,activation="relu")(x)

x = layers.Dense(units=1,name = "dense_final")(x)
x = layers.Activation("sigmoid")(x)

b_vgg_model = tf.keras.Model(b_vgg.input, [x])

In [ ]:
X_train_b_vgg = tf.keras.applications.vgg16.preprocess_input(X_train_binary, data_format=None)
X_test_b_vgg = tf.keras.applications.vgg16.preprocess_input(X_test_binary, data_format=None)

In [ ]:
adam = tensorflow.keras.optimizers.Adam(learning_rate = 0.0001)
b_vgg_model.compile(optimizer=adam ,loss='binary_crossentropy', metrics=['accuracy'])
b_vgg_model.fit(X_train_b_vgg, Y_train_binary, batch_size=16, epochs=5)

Epoch 1/5
215/215 [==============================] - 69s 247ms/step - loss: 0.7090 - accuracy: 0.6823
Epoch 2/5
215/215 [==============================] - 52s 241ms/step - loss: 0.3752 - accuracy: 0.8382
Epoch 3/5
215/215 [==============================] - 51s 239ms/step - loss: 0.2289 - accuracy: 0.9107
Epoch 4/5
215/215 [==============================] - 51s 239ms/step - loss: 0.1378 - accuracy: 0.9433
Epoch 5/5
215/215 [==============================] - 52s 240ms/step - loss: 0.1253 - accuracy: 0.9520


In [ ]:
score = b_vgg_model.evaluate(X_test_b_vgg,Y_test_binary)

27/27 [==============================] - 10s 268ms/step - loss: 0.1477 - accuracy: 0.9523


In [ ]:
b_vgg_model.save('/content/drive/My Drive/MRI Classification/VGG16_Binary.h5')

In [ ]:
b_vgg_model = tf.keras.models.load_model('/content/drive/My Drive/MRI Classification/VGG16_Binary.h5')

In [ ]:
y = b_eff_model.get_layer('dense_final').output
y = tf.math.scalar_mul(0.2, y)
y = layers.Activation("sigmoid")(y)

b_vgg_y_model = Model(inputs=b_eff_model.input, outputs=y)

In [ ]:
y_pred_vgg = b_vgg_model.predict(X_test_binary)
y_pred_vgg = (y_pred_vgg > 0.5)
    
cm = confusion_matrix(Y_test_binary, y_pred_vgg)

print(cm)

[[308  98]
 [ 89 365]]


In [ ]:
accuracy = accuracy_score(y_pred_vgg, Y_test_binary)
print('Accuracy: ' ,accuracy)
precision = precision_score(y_pred_vgg, Y_test_binary, average = None)
print('Precision: ' , precision)
recall = recall_score(y_pred_vgg, Y_test_binary,average = None)
print('Recall: ' ,recall)
f1 = f1_score(y_pred_vgg, Y_test_binary,average = None)
print('F1 score: ',f1)

Accuracy:  0.7825581395348837
Precision:  [0.75862069 0.80396476]
Recall:  [0.77581864 0.78833693]
F1 score:  [0.76712329 0.79607415]
